In [ ]:
from marketplace.app.v0_0_1.hpc_app import HpcGatewayApp
import os

# Setting the client_id of the hpc app and pass the access_token to using the app through SDK
hpc = HpcGatewayApp(client_id="<client_id>",
                    access_token=os.environ.get('MP_ACCESS_TOKEN'))

In [ ]:
# The heartbeat capability to show the state of server and the HPC resource
print(hpc.heartbeat())

## initialize a new job

using capability `new_trasformation`, the app create a folder under the user repository path in the remote HPC resource. 
The remote path name is returned for futher files transfor operations and as `transformation_id` for job submission.

In [ ]:
# The resource_id returned will be the remote path
resourceid = hpc.new_job(config=None)
print(resourceid)

In [ ]:
!ssh daint.cscs.ch "cd /scratch/snx3000/jyu/firecrest/jusong_yu/ && ls -l"

# Upload input files and slurm job script to the path

The capabilities `updateDataset`, `getDataset` and `deleteDataset` are for upload, download and delete files on remote path.

However, instead of remote path only it is also require to pass filename to manipulate the files which is not completely compatible with the initial designs of these capabilities which are for database in the first place. 
I hack it a bit to pass the filename by a json object.

In [ ]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{resourceid}'
!ssh daint.cscs.ch $cmd

In [ ]:
! head -n 10 ./sample_files/colloid.in

In [ ]:
# send LAMMPS input file and job script to remote folder
hpc.upload(resourceid=resourceid, source_path='./sample_files/colloid.in')
hpc.upload(resourceid=resourceid, source_path='./sample_files/submit.sh')

In [ ]:
cmd = f'ls -l /scratch/snx3000/jyu/firecrest/jusong_yu/{resourceid}'
!ssh daint.cscs.ch $cmd

In [ ]:
# download the file
content = hpc.download(resourceid=resourceid, filename='submit.sh')
print(content)

## Submit job and check the jobs list

`start_transformation` corresponding to capability `startTransformation` which will do actuall job submit into the scheduler system and return the job info.
Then I can using `get_transformation_list` to retrieve all the jobs of current user in remote resource.


In [ ]:
import json

jblst = hpc.list_job()
print(json.dumps(jblst, indent=2))

In [ ]:
rsp = hpc.run_job(resourceid=resourceid)
print(rsp)

In [ ]:
jblst = hpc.list_job()
print(jblst)

In [ ]:
# resource_id = 'f1701e07-7e48-4b53-a915-6e0a9caad4cf'
cmd = f"cat /scratch/snx3000/jyu/firecrest/jusong_yu/{resourceid}/log.lammps"
!ssh daint.cscs.ch $cmd

In [ ]:
# download the file
content = hpc.download(resourceid=resourceid, filename='log.lammps')
print(content)